In [1]:
#!pip install --upgrade oct2py
import numpy as np
from oct2py import octave
import matplotlib.pyplot as plt
import os
from scipy.optimize import root
import math

`DyPy` is a class that we create and store in a file called `MyPy.py`. It contains methods for 

* converting data from a Dynare stucture to Numpy arrays.
* analyzing the data. 

See the file for details. 

In [2]:
from MyPy import DyPy

Creates a file called `t_call.m`. This informs Dynare of the format we want for output. It is put here so that adjusments can be made easily; for example the number of periods can be changed here rather than by looking at the `.mod` file. It is included in the `.mod` file via: `@#include "t_call.m"` in `model_1.mod`

* `order`: the order of approximation for the stochastic simulation. order=1 is a fsrst-order approximation (linearization) Higher-order approximations (e.g., order=2 for second-order) capture nonlinear effects in the model, but they require more computational resources.
* `periods`: number of periods (time steps) for which the simulation will be run. periods=1000 means that the model will be simulated for 1000 periods. The default value is 0, which means no simulation will be performed.
* `nograph`: suppresses the graphical output of the simulation results in Dynare since we draw them separatetly. 
* `irf`: the number of periods for which the impulse response functions (IRFs) will be computed and plotted. 

In [3]:
with open('t_call.m', 'w') as file:
    file.write("stoch_simul(nograph, order=1, periods=1000, irf=30);")

 `make_model.ipyn` writes a file called `t_model.m`. `t_model.m`is the `model` block in the Dynare `.mod` file. It can instead be created directlty by commenting out the next `%run` line and coding the model. `make_model.ipyn` simplifies model creation when many of the lines are very similar to each other. Then they can be written in for loops. For the model we are currently working with, there is not much advantage to writing the file via `make_model` but I have done so nonetheless. 
 
 The created file is included in the `.mod` file via `@#include "t_model.m"` in `model_1.mod`. 

In [4]:
%run make_model.ipynb

`make_files.ipynb` writes the other Dynare blocks. See that file for details. However, note that in doing so, it solves for the steady state of the model using `root` from `scipy`. If the results below do not indicate success, the `.mod` file may not run correctly. Also, note that though we solve for the steady state here, we do not give Dynare the steady state but rather use these as starting values for their ss solver. I will later code a verification the steady states are the same. 

In [5]:
%run make_files_1.ipynb

eps,tht,alf,bet,om_0,om_1,om_2,delt,psi_n,psi_k,a_0,a_1,a_2
eps,tht,alf,bet,om_0,om_1,om_2,delt,psi_n,psi_k,var_a_0,var_a_1,var_a_2,rho_a_0,rho_a_1,rho_a_2


Tells octave where to find the .m file it will run. The file is `call_dynare`. Set to wherever you have this file. 




In [6]:
#a=octave.addpath(os.path.expanduser('C:/Users/blankenw/GitHub/dynare_progs/np_1'))

Makes a call to Octave, which makes a call to Dynare. Then the `.mod` file is run in Dynare. Output from Dynare is displayed as it runs.

In [7]:
# Create an empty dictionary to store your variables
variables =['t0','t1','t2']


In [8]:

# A list of dictionaries representing updates to apply to my_dict:
new_pars = [{'psi_n': 0, 'psi_k': 0}, 
           {'psi_n': 2, 'psi_k': 0}, 
           {'psi_n': 0, 'psi_k': 2}]

# Create an empty dictionary to store your variables
t=[]



for i, update in enumerate(new_pars):
    model_params.update(update)
    %run make_files_2.ipynb
    y=1
    x=octave.call_dynare(y,nout=1)
    oo = octave.pull('oo_') 
    instance = DyPy(oo)
    t.append(instance)


 message: The solution converged.
 success: True
  status: 1
     fun: [-1.110e-16  0.000e+00 ...  8.169e-13 -9.029e-13]
       x: [ 4.368e-01  2.816e-01 ...  8.740e-01  4.816e-01]
    nfev: 42
    fjac: [[-5.560e-02 -5.969e-11 ... -1.278e-03 -5.232e-03]
           [-3.941e-02  3.857e-11 ...  5.439e-02  5.471e-04]
           ...
           [-3.118e-01 -2.372e-01 ... -2.009e-01 -2.498e-01]
           [ 6.222e-02 -6.227e-02 ... -2.961e-04  1.741e-03]]
       r: [-1.799e+01  2.783e-01 ...  4.606e-01  1.500e-01]
     qtf: [-9.314e-13 -1.446e-10 ...  3.518e-12 -1.464e-13]
    call_dynare at line 4 column 4
    _pyeval at line 57 column 30

    call_dynare at line 4 column 4
    _pyeval at line 57 column 30

Starting Dynare (version 6.1).
Calling Dynare with arguments: none
Starting preprocessing of the model file ...
Found 17 equation(s).
Evaluating expressions...
Computing static model derivatives (order 1).
Normalizing the static model...
Finding the optimal block decomposition of the sta

Dynare creates a `structure` which we will need so we `pull` it.

The output from Dynare is stored in a file called `oo_`. This brings `oo_` in as `oo`.

Draw all impulse response functions. 

In [ ]:
def plot_dta(instances, attributes,periods=30):
    plt.figure(figsize=(15, 5))
    for j, attribute in enumerate(attributes):
        # Create a subplot for this attribute
        plt.subplot(1, len(attributes), j+1)
        for i, instance in enumerate(instances):
            # Get the attribute value (time series data)
            attr_value = getattr(instance, attribute)
            attr_value = attr_value.reshape(-1, 1)
            length = attr_value.shape[0]
            tm = np.linspace(1, length, length)
            # Add a line to the plot for this time series
            plt.plot(tm[0:periods], attr_value[0:periods], label=labels[i])

        plt.title(attribute)
        plt.legend()

    # Show the figure with all subplots


In [ ]:
labels = ['No adjustment cost', 'Adjustment cost on labor', 'Adjustment cost on capital']

all_attributes =[['ir_n_0_a_0_shock', 'ir_n_1_a_0_shock', 'ir_n_2_a_0_shock'],
                 ['ir_s_0_a_0_shock', 'ir_s_1_a_0_shock', 'ir_s_2_a_0_shock'],
                 ['ir_k_a_0_shock', 'ir_i_a_0_shock'],
                    ['ir_k_0_a_1_shock', 'ir_k_1_a_1_shock', 'ir_k_2_a_1_shock'],
                    ['ir_c_0_a_1_shock', 'ir_c_1_a_1_shock', 'ir_c_2_a_1_shock'],
                    ['ir_k_a_1_shock', 'ir_i_a_1_shock'],
                    ['ir_k_0_a_2_shock', 'ir_k_1_a_2_shock', 'ir_k_2_a_2_shock'],
                    ['ir_c_0_a_2_shock', 'ir_c_1_a_2_shock', 'ir_c_2_a_2_shock'],
                    ['ir_k_a_2_shock', 'ir_i_a_2_shock']]
for attributes in all_attributes:
    plot_dta(t, attributes,periods=30)
    plt.tight_layout()
    plt.show()                  

In [ ]:
t[0].oo_dict

In [ ]:
import pickle

In [ ]:
a=[t[0].oo_dict,t[1].oo_dict,t[2].oo_dict]

In [ ]:
a=[1,2,3]

In [ ]:
class DPy:
    def __init__(self,x):
        self.outer_attr = x
        self.inner = self.InnerClass()
        self.oo_dict = {key: np.array(value) for key, value in x.items()}   
    def ex_1(self):
        print(self.outer_attr)
    
    class InnerClass:
        def __init__(self):
            self.inner_attr = "Inner Attribute"
        
        def inner_method(self):
            print("Inner Method")

In [ ]:
tst=DPy(t[0])

In [ ]:
class DyPy:
    def __init__(self, x):
        """Initializes the class with a dictionary of arrays."""
        self.oo_dict = {key: np.array(value) for key, value in x.items()}
        """Converts the dictionary of arrays into attributes of the class."""
        for key, value in self.oo_dict.items():
            setattr(self, key, value)
        dict_irfs = self.irfs.item()
        self.dict_irfs = {key: np.array(value) for key, value in dict_irfs.items()}
        for key, value in self.dict_irfs.items():
            if key!='list':
                setattr(self, 'ir_'+ key, value)
        self.list_irfs = self.list_irfs()
        
        
        
        
        
        

In [ ]:
a=DyPy(t[0].oo_dict)